In [5]:
# Install python-chess if not already installed
!pip install python-chess
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [27]:
import chess.pgn
import numpy as np
import pandas as pd
import chess
import chess.engine

In [21]:
# Function to parse PGN file and extract moves
def parse_pgn(pgn_file):
    games = []
    with open(pgn_file) as f:
        while True:
            game = chess.pgn.read_game(f)
            if game is None:
                break
            moves = []
            node = game
            while not node.is_end():
                next_node = node.variation(0)
                move = node.board().san(next_node.move)
                moves.append(move)
                node = next_node
            games.append(moves)
    return games


In [25]:
# Load and parse the PGN file
games = parse_pgn('Carlsen.pgn')



In [27]:
# Convert moves to a dataframe for easier handling
df = pd.DataFrame(games)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,338,339,340,341,342,343,344,345,346,347
0,d4,Nf6,Nf3,d5,e3,Bf5,c4,c6,Nc3,e6,...,None,None,None,None,None,None,None,None,None,None
1,e4,Nf6,e5,Nd5,d4,d6,Nf3,Bg4,Bc4,e6,...,None,None,None,None,None,None,None,None,None,None
2,e4,e6,d4,d5,Nc3,Bb4,e5,Ne7,a3,Bxc3+,...,None,None,None,None,None,None,None,None,None,None
3,d4,d5,c4,e6,Nc3,Nf6,e3,Be7,Nf3,O-O,...,None,None,None,None,None,None,None,None,None,None
4,e4,c5,Nf3,Nc6,d4,cxd4,Nxd4,e5,Nb5,d6,...,None,None,None,None,None,None,None,None,None,None


In [29]:
# Function to convert board to a matrix
def board_to_matrix(board):
    piece_map = board.piece_map()
    matrix = np.zeros((8, 8), dtype=np.int8)
    for square, piece in piece_map.items():
        matrix[square // 8, square % 8] = piece.piece_type * (1 if piece.color == chess.WHITE else -1)
    return matrix


In [31]:
# Generate input and output pairs for training
X, y = [], []
for moves in games:
    board = chess.Board()
    for move in moves:
        X.append(board_to_matrix(board))
        board.push_san(move)
        y.append(board_to_matrix(board))




In [33]:
# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

In [35]:
# Reshape for CNN input
X = X.reshape(-1, 8, 8, 1)
y = y.reshape(-1, 8, 8, 1)

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, Input

In [43]:
# Build the CNN model
model = Sequential([
    Input(shape=(8, 8, 1)),  
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(8 * 8 * 13, activation='softmax'),
    Reshape((8, 8, 13))
])

In [53]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [57]:
# Train the model
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
17390/17390 ━━━━━━━━━━━━━━━━━━━━ 103s 6ms/step - accuracy: 0.8174 - loss: 0.5725
Epoch 2/10
17390/17390 ━━━━━━━━━━━━━━━━━━━━ 100s 6ms/step - accuracy: 0.8764 - loss: 0.3883
Epoch 3/10
17390/17390 ━━━━━━━━━━━━━━━━━━━━ 101s 6ms/step - accuracy: 0.8903 - loss: 0.3497
Epoch 4/10
17390/17390 ━━━━━━━━━━━━━━━━━━━━ 101s 6ms/step - accuracy: 0.8976 - loss: 0.3298
Epoch 5/10
17390/17390 ━━━━━━━━━━━━━━━━━━━━ 110s 6ms/step - accuracy: 0.9024 - loss: 0.3169
Epoch 6/10
17390/17390 ━━━━━━━━━━━━━━━━━━━━ 104s 6ms/step - accuracy: 0.9056 - loss: 0.3083
Epoch 7/10
17390/17390 ━━━━━━━━━━━━━━━━━━━━ 108s 6ms/step - accuracy: 0.9083 - loss: 0.3007
Epoch 8/10
17390/17390 ━━━━━━━━━━━━━━━━━━━━ 108s 6ms/step - accuracy: 0.9100 - loss: 0.2959
Epoch 9/10
17390/17390 ━━━━━━━━━━━━━━━━━━━━ 107s 6ms/step - accuracy: 0.9115 - loss: 0.2917
Epoch 10/10
17390/17390 ━━━━━━━━━━━━━━━━━━━━ 106s 6ms/step - accuracy: 0.9127 - loss: 0.2886


In [69]:
# Save the model with the optimizer configuration
model.save('chess_cnn_model.keras')



In [1]:
from keras.models import load_model

# Load the model from the file in native Keras format
model = load_model('chess_cnn_model.keras')


In [31]:
# Function to convert board to a matrix
def board_to_matrix(board):
    piece_map = board.piece_map()
    matrix = np.zeros((8, 8), dtype=int)
    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        matrix[row][col] = piece.piece_type
    return matrix

In [33]:
# Function to convert matrix back to a chess board
def matrix_to_board(matrix):
    board = chess.Board()
    board.clear_board()
    for i in range(8):
        for j in range(8):
            if matrix[i][j] != 0:
                piece = chess.Piece(matrix[i][j], chess.WHITE if matrix[i][j] > 0 else chess.BLACK)
                board.set_piece_at(chess.square(j, i), piece)
    return board

In [35]:
# Function to find the best move predicted by the model
def find_best_move(board, predicted_matrix):
    legal_moves = list(board.legal_moves)
    best_move = None
    max_confidence = -1
    
    for move in legal_moves:
        board.push(move)
        move_matrix = board_to_matrix(board).reshape(1, 8, 8, 1)
        board.pop()
        
        confidence = np.max(predicted_matrix)
        if confidence > max_confidence:
            max_confidence = confidence
            best_move = move
            
    return best_move

In [37]:
# Function to make a move using the model
def make_move(board, model):
    board_matrix = board_to_matrix(board).reshape(1, 8, 8, 1)
    predicted_matrix = model.predict(board_matrix)
    best_move = find_best_move(board, predicted_matrix)
    move_san = board.san(best_move)
    board.push(best_move)
    return move_san

In [39]:
# Function to check if a move is legal
def is_legal_move(board, move):
    try:
        move_obj = board.parse_san(move)
        if move_obj in board.legal_moves:
            return True
        else:
            return False
    except:
        return False

In [41]:
# Function to handle ambiguous moves
def handle_ambiguous_move(board, move):
    try:
        board.push_san(move)
        return True
    except chess.AmbiguousMoveError:
        print(f"Ambiguous move: {move}. Please specify the move more clearly.")
        return False
    except ValueError:
        print(f"Illegal move: {move}. Please try again.")
        return False

In [43]:
import chess
import chess.engine

# Function to play against Stockfish
def play_against_stockfish(board, model, stockfish_path):
    engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)
    
    while not board.is_game_over():
        print(board)
        user_choice = input("Enter 'user' to make a move, 'stockfish' to let Stockfish make a move, or 'abort' to end the game: ").strip().lower()
        
        if user_choice == 'abort':
            print("Game aborted by user.")
            break
        
        if user_choice == 'user':
            user_move = input("Your move: ")
            while not handle_ambiguous_move(board, user_move):
                user_move = input("Your move: ")
                if user_move.lower() == 'abort':
                    print("Game aborted by user.")
                    return
        
        elif user_choice == 'stockfish':
            result = engine.play(board, chess.engine.Limit(time=1.0))
            board.push(result.move)
            print(f"Stockfish move: {board.san(result.move)}")
        
        else:
            print("Invalid choice. Please enter 'user', 'stockfish', or 'abort'.")
            continue
        
        if board.is_game_over():
            break
        
        model_move = make_move(board, model)
        print(f"Model move: {model_move}")
    
    engine.quit()

In [51]:
# Play a game against Stockfish
# board = chess.Board()
# stockfish_path = "path/to/stockfish"  # Replace with the actual path to the Stockfish executable
# play_against_stockfish(board, model, stockfish_path)

In [49]:
# Play a game against the model
board = chess.Board()
while not board.is_game_over():
    print(board)
    user_move = input("Your move (or type 'abort' to quit): ")
    
    if user_move.lower() == 'abort':
        print("Game aborted.")
        break
    
    while not handle_ambiguous_move(board, user_move):
        user_move = input("Your move (or type 'abort' to quit): ")
        if user_move.lower() == 'abort':
            print("Game aborted.")
            break
    
    if board.is_game_over():
        break
    
    model_move = make_move(board, model)
    print(f"Model move: {model_move}")
    print(board)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


Your move (or type 'abort' to quit):  e4


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Model move: Nh6
r n b q k b . r
p p p p p p p p
. . . . . . . n
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
r n b q k b . r
p p p p p p p p
. . . . . . . n
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


Your move (or type 'abort' to quit):  Nc3


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Model move: Rg8
r n b q k b r .
p p p p p p p p
. . . . . . . n
. . . . . . . .
. . . . P . . .
. . N . . . . .
P P P P . P P P
R . B Q K B N R
r n b q k b r .
p p p p p p p p
. . . . . . . n
. . . . . . . .
. . . . P . . .
. . N . . . . .
P P P P . P P P
R . B Q K B N R


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Model move: Rg8
r n b . k b r .
p p B p p n p p
. . . . . . . .
. . . . . . . .
. . . P P . . .
. . . . . Q . .
P P P . . P P P
R . . . K . N R
r n b . k b r .
p p B p p n p p
. . . . . . . .
. . . . . . . .
. . . P P . . .
. . . . . Q . .
P P P . . P P P
R . . . K . N R


Your move (or type 'abort' to quit):  Nh3


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Model move: Rh8
r n b . k b . r
p p B p p n p p
. . . . . . . .
. . . . . . . .
. . . P P . . .
. . . . . Q . N
P P P . . P P P
R . . . K . . R
r n b . k b . r
p p B p p n p p
. . . . . . . .
. . . . . . . .
. . . P P . . .
. . . . . Q . N
P P P . . P P P
R . . . K . . R


Your move (or type 'abort' to quit):  abort


Game aborted.
